# Visual Odometry (VO)

In this assignment, you do not need. a GPU. You will use the pykitti module and KITTI odometry dataset.

You can download the odometry data from [here](https://drive.google.com/file/d/1PJOUnM3nEwDpqiRvfqUnkNPJZpM4PKYV/view?usp=sharing).

## Monocular VO with OpenCV on KITTI

For each consecutive frame pair in the sequence, you will compute the relative pose between the frames and visualize it. You will use:

* pykitti code similar to what you wrote in mvs part to load the sequence with ground-truth info. (Check out the [demo code](https://github.com/utiasSTARS/pykitti/blob/master/demos/demo_odometry.py))
* OpenCV functions to compute and visualize the features and the essential matrix.

Please follow these steps to complete the assignment:

1. You can use the ORB Feature to do the feature matching:
    `orb = cv2.ORB_create()` to create the ORB object
    and then `orb.detectAndCompute()` to find the keypoints and descriptors on both frames

2. You can use brute-force matcher to match ORB descriptors:
    `bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)`

3. After matching the descriptors, sort the matched keypoints.

4. Draw matches on the two images using the `cv2.drawMatches()` function.

5. Compute the essential matrix using the `cv2.findEssentialMat()` function. Note that you need the matching points and the instrinsics for this function. 

6. Extract the rotation and translation from the essential matrix using the `cv2.recoverPose()` function.

7. Multiply the estimated rotation and translation with the previous rotation and translation. Initialize rotation to identity and translation to zeros on the first frame.

8. Display the current image with the keypoints on it using the `cv2.drawKeypoints()` function.

9. Update the previous rotation and translation as the current rotation and translation.

10. Draw the estimated trajectory as blue and ground-truth trajectory as green. You can use the `cv2.circle()` function.


You can create a video of your visualization of images and poses for the provided sequence.

**Bonus**: Compute the absolute trajectory error between the estimated trajectory and the ground-truth trajectory. 

Some examples repositories that might be useful:
* https://bitbucket.org/castacks/visual_odometry_tutorial/src/master/visual-odometry/
* https://github.com/uoip/monoVO-python


---

In [1]:
import pykitti
import cv2
import numpy as np
import math
from PIL import ImageOps

# Load in Kitti Odometry data
basedir = '../KITTI_odometry' # Change this to the directory where you store KITTI data
sequence = '09' # Change this to the sequence you would like to use

dataset = pykitti.odometry(basedir, sequence)

# dataset.calib:      Calibration data are accessible as a named tuple
# dataset.timestamps: Timestamps are parsed into a list of timedelta objects
# dataset.poses:      List of ground truth poses T_w_cam0
# dataset.camN:       Generator to load individual images from camera N
# dataset.gray:       Generator to load monochrome stereo pairs (cam0, cam1)
# dataset.rgb:        Generator to load RGB stereo pairs (cam2, cam3)
# dataset.velo:       Generator to load velodyne scans as [x,y,z,reflectance]

In [2]:
### KITTI parameters ###
fx = 718.86 # focal 
cx = 607.19 # cx offset
cy = 185.22 # cy offset

# len_traj = len(dataset.timestamps)
len_traj = 650

trajMap = np.zeros((len_traj, len_traj, 3), dtype=np.uint8)

prev_img = None
for i, (cur_img, gt_pose) in enumerate(zip(dataset.cam2, dataset.poses)):
    
    # convert PIL image to grasycale
    cur_img = ImageOps.grayscale(cur_img)
    # convert PIL image to cv2 image format
    cur_img = np.array(cur_img) 
    # cur_img = cur_img[:, :, ::-1].copy() 
    # cur_img = cv2.cvtColor(cur_img, cv2.COLOR_BGR2RGB)
    
    if i == 0: # first frame of the sequence
        # Initialize R and t matrices for estimated poses
        curr_R = np.eye(3)
        curr_t = np.array([0, 0, 0])
        
        prev_img = cur_img # set current frame as the prev_img for the next frame

    else: # not the first frame of the sequence
        #====================== Use ORB Feature to do feature matching =====================#
        # create ORB features
        orb = cv2.ORB_create(nfeatures=6000)

        # find the keypoints and descriptors with ORB
        kp1, des1 = orb.detectAndCompute(prev_img, None)
        kp2, des2 = orb.detectAndCompute(cur_img, None)

        # use brute-force matcher
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # Match ORB descriptors
        matches = bf.match(des1, des2)

        # Sort the matched keypoints in the order of matching distance
        # so the best matches came to the front
        matches = sorted(matches, key=lambda x: x.distance)

        img_matching = cv2.drawMatches(prev_img, kp1, cur_img, kp2, matches[0:100], None)
        cv2.imshow('Feature Matching', img_matching)

        pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
        pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

        # compute essential matrix
        E, mask = cv2.findEssentialMat(pts1, pts2, focal=fx, pp=(cx, cy), method=cv2.RANSAC, prob=0.999, threshold=1)
        pts1 = pts1[mask.ravel() == 1]
        pts2 = pts2[mask.ravel() == 1]
        _, R, t, mask = cv2.recoverPose(E, pts1, pts2, focal=fx, pp=(cx, cy))

        # get estimated camera motion
        R = R.transpose()
        t = -np.matmul(R, t)
        
        if i == 1:
            curr_R = R
            curr_t = t
            
        else:
            curr_R = np.matmul(prev_R, R)
            curr_t = np.matmul(prev_R, t) + prev_t

        # draw the current image with keypoints
        curr_img_kp = cv2.drawKeypoints(cur_img, kp2, None, color=(0, 255, 0), flags=0)
        cv2.imshow('Keypoints From Current Image', curr_img_kp)

        # update prev_img with cur_img
        prev_img = cur_img

    prev_R = curr_R
    prev_t = curr_t
    
    # draw estimated trajectory (blue) and gt trajectory (red)
    offset_draw = (int(len_traj/2))
    cv2.circle(trajMap, (int(curr_t[0])+offset_draw, int(curr_t[2])+offset_draw), 1, (255,0,0), 2)
    # draw gt trajectory
    cv2.circle(trajMap, (int(gt_pose[0:3, 3][0])+offset_draw, int(gt_pose[0:3, 3][2])+offset_draw), 1, (0,255,0), 2)
    cv2.imshow('Trajectory', trajMap)
    cv2.waitKey(1)

cv2.imwrite('trajMap.png', trajMap)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/cangozpi/miniconda3/envs/carla/lib/python3.7/site-packages/cv2/qt/plugins"
QObject::moveToThread: Current thread (0x5591a1cfc6e0) is not the object's thread (0x5591a27e9870).
Cannot move to target thread (0x5591a1cfc6e0)

QObject::moveToThread: Current thread (0x5591a1cfc6e0) is not the object's thread (0x5591a27e9870).
Cannot move to target thread (0x5591a1cfc6e0)

Qt: Session management error: Could not open network socket
QObject::moveToThread: Current thread (0x5591a1cfc6e0) is not the object's thread (0x5591a27e9870).
Cannot move to target thread (0x5591a1cfc6e0)

QObject::moveToThread: Current thread (0x5591a1cfc6e0) is not the object's thread (0x5591a27e9870).
Cannot move to target thread (0x5591a1cfc6e0)

QObject::moveToThread: Current thread (0x5591a1cfc6e0) is not the object's thread (0x5591a27e9870).
Cannot move to target thread (0x5591a1cfc6e0)

QObject::moveToThread: Current thread (0x5591a1cfc6e0) is

True

: 